## Library and data imports##

In [55]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import pandas as pd
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# review = "Please note that this is purely my experience of the game during the stupid hours I have sunk into it. I won't be completely critical of the game and I'll give credit to a couple of things, most notably the gameplay. I like the animations and the motion of the players, they are fun to control. I play on normal gameplay speed and it feels great. Players are responsive and it is very fun trying to keep shape defensively (especially against ultimate difficulty) but with practice, you get there. Attacking is satisfying too, I like the different dribbling styles, although R1 dribbling at the time of this review is a little overpowered, but even still, I am happy that it's possible to dribble past players in this year's game and score some lovely solo goals. I play purely offline, as where I am currently located I just cannot get a good enough connection to have any kind of online experience. HOWEVER - despite EA releasing updates, not one of them has combated the biggest issue myself and many MANY others face, which is the abhorrent optimisation which is leading to stutters and for some, lag and game crashes. I would not want to know how many of those hours I have spent trialling fixes and it is the first time in my gaming experience to encounter a game that just won't play. Sometimes a workaround will fix things temporarily, but upon the next launch, it just doesn't work again. This to me is unacceptable. They are aware of the problem, yet nothing has changed now for well over a month of it being released. The hell I've been through to sort it should never be required to play any game. I paid £80, but I feel like EA owe me a year's salary for the amount of work I've done on their game for them. Crooks. Stutters in these kinds of games destroy any pleasurable experience you can get from it. Imagine playing an F1 game, but before every corner it stutters for 3 seconds and when it fixes itself, you've crashed into an entire neighbouring village and caused £500 million worth of damage, you'd say 'Forget it' and toss it in the bin. The same goes for football games. You need precision and reactions because timing is important. You can't do that with 3-second stutters intermittently during a match. How can nobody have seen this? I feel like Mugatu from Zoolander. It was a new era for the franchise, a fresh start, so I allowed myself to believe that things may be different, but it just isn't the case for any of the offline modes. If you think adding cut scenes of players winning the Balon d'Or (where nobody even speaks other than some announcer, which freaks me out) and an open-top bus parade of trophies you've won (which is cool the first time, but it is the same every single time after) then you don't understand why we play football games - WE WANT TO PLAY, NOT SIT THERE AND WATCH SILENT MOVIES. Is this their attempt at making it more immersive? Disaster. True, some promising ideas were introduced, like tactical vision and hiring coaches in manager mode, but here's the thing - that was bugged when it was released. Imagine working on a new feature, but for it not to work properly. What I mean is, that you can't fire coaches, you'll lock the game and have no chance to back out. I actually hope it doesn't get patched, because it doesn't matter, it tells us what we already know - they don't care. It's a shame because I think that if they'd just put more effort into it, this could've been a really good game. The foundation is there with the gameplay, and that's the heart of any good football game. But why not just stop being a bunch of fraggles and develop your offline game modes more? The amount of work they put into 'create a team' for example, which was just uninspiring, is what I am talking about. Put that work into actually making a comprehensive, immersive career mode. 'Create a team' was fantastic in the F1 game, it works in that circumstance, but for a manager mode on a football game? It isn't what we want. For those who can't be bothered to read all that: you'll feel just like Mugatu from Zoolander - It really is the same face."
# review2 = 'MWIII is the worst campaign of all time. 14 short missions and half are just spec ops missions from MWII, dropping you in a section of the warzone map and giving you objectives. It was like a warzone tutorial. The story is also far less than adequate. This seems like it should have been a DLC for MWII . I have never been more upset with a game. The multiplayer needs to be phenomenal for this game to be worth buying at all.'

# reviews = [review, review2]
df = pd.read_csv('dataset.csv')
reviews = df['review']
y_pre = df['voted_up']


## Dealing with imbalanced data ##
This would be where we deal with the fact that most of the reviews are positive and will probably do subsampling

## Tokenization ##
Can cover the following hyperparameters (and their tuning):
- How do we deal with contractions?
- What kind of normalization do we do? (convert accents to ascii? stemming? lemmatization?)
- Do we use stopwords? Which ones?
- Other CountVectorizer hyperparams
- How do we handle punctuation?  
- How much should we subsample? (Need to determine performance metric first)

In [47]:
# preprocessing so the model can understand the reviews
pp_reviews = []
y = []

# filters out reviews with ascii characters -- we get an error if we don't include this. Need to narrow down the exact cause if possible.
for i in range(0, len(reviews)-1):
    if str(reviews[i]).isascii():
        pp_reviews.append(str(reviews[i]))
        y.append(y_pre[i])

# tokenizes the reviews
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(np.array(pp_reviews))

# transforms them so we deal with term frequencies rather than term counts
tfidf_transformer = TfidfTransformer()
X = tfidf_transformer.fit_transform(X_train_counts)

## Model Training ##

Will need to have consideration of how we measure performance (it likely isn't pure accuracy. Precision? Recall?)

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

sgd_classifier = SGDClassifier(random_state=42)
sgd_classifier.fit(X_train, y_train)
y_pred = sgd_classifier.predict(X_test)
print(accuracy_score(y_test, y_pred))


0.979632740884841
